## Generates Labels for Binary Classification
    
    - probability of an alarm in the next 5 minutes (original label)
    - simultaneously creates labels for 1 hour and 24 hours
    - keeps track of number of missing data points in the past data
    - filters data points where there are missing data in the last T-days (T=7)

In [ ]:
import datetime
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import pickle

In [ ]:
inv_df = pd.read_csv('all_inverters.csv')
# inv_df.head()

target_codes = [7006, 3511, 7502, 7501, 3504, 6448, 1500, 7704]
alarm_df = pd.read_csv('all_alarms.csv')
alarm_df = alarm_df[alarm_df["Error Code"].isin(target_codes)]
alarm_df = alarm_df[(alarm_df.hod >= 6) & (alarm_df.hod <= 18)]  # original (6,17)
print(alarm_df.shape)
inverters = sorted(alarm_df["Controller Name"].unique().tolist())

# alarm_df.head()

In [ ]:
alarm_df.head()

## Generate Labels for all Inverters

In [ ]:
def impute_data(df_original, colname, method='backward'):
    """
        Data imputation based on past/future data for the same month/day/hour/minute
        if method = 'backward' then only past data are used
        if method = 'all' then all data (past and future) are used
    """

    df_im = df_original.copy()
    count_imputed = 0
    missing_indices = df_original[df_original[colname].isna()].index
    print(f"Initially {len(missing_indices)} missing data points")
    filled_dates = []
    
    for ii in tqdm(missing_indices):
    # for ii, row in tqdm(df_full.iterrows()):
        row = df_full.iloc[ii]
    #     print(df_im['power'].isna().sum())
        if np.isnan(row[colname]):
            if method == 'backward':
                # only use data till now
                df_ = df_full[:ii].copy()
                df_ = df_[~df_[colname].isna()]
            else:
                df_ = df_full.copy()
                df_ = df_[~df_[colname].isna()]
            if df_.shape[0] == 0:
                continue
            try1 = df_[(df_.month==row['month']) & (df_.day==row['day']) & (df_.hour==row['hour']) & (df_.minute==row['minute'])]
            if try1.shape[0] > 0:            
                imputed_value = try1[colname].mean()
                df_im.loc[ii, colname] = imputed_value
                filled_dates.append(row['date'])
                count_imputed += 1
                continue
            else:
                try2 = df_[(df_.month==row['month']) & (df_.day==row['day']) & (df_.hour==row['hour'])]
                if try2.shape[0] > 0:
                    imputed_value = try2[colname].mean()
                    df_im.loc[ii, colname] = imputed_value
                    filled_dates.append(row['date'])
                    count_imputed += 1
                    continue
                else:
                    try3 = df_[(df_.month==row['month']) & (df_.day==row['day'])]
                    if try3.shape[0] > 0:
                        imputed_value = try3[colname].mean()
                        df_im.loc[ii, colname] = imputed_value
                        filled_dates.append(row['date'])
                        count_imputed += 1
                        continue
    print(count_imputed, df_original['power'].isna().sum(), df_im['power'].isna().sum())
    return df_im, filled_dates




In [ ]:
TIMESTAMP_COL_NAME = 'date'
POWER_COL = 'power'
min_past_days = 7
feature_dfs = dict()
impute_data = False
label_col = 'label_1h'

# for inverter in ['INV-04']:
for inverter in inverters:
    features = ['IN.GMRX.CHAR.'+inverter+'.Active Power (kW)', 
    #             'IN.GMRX.CHAR.WS-20 MW.Ambient Temperature (°C)', # all nulls
                'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)',
                'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)',
    #             'IN.GMRX.CHAR.WS-5 MW.Ambient Temperature (°C)',  # all nulls
                'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)',
                'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)'
               ]
    columns = ['date'] + features
    alarm_df_i = alarm_df[alarm_df["Controller Name"] == inverter].copy()
    alarm_df_i['Raised Time'] = pd.to_datetime(alarm_df_i['Raised Time'])
    alarm_df_i = alarm_df_i.sort_values('Raised Time').reset_index(drop=True)
    alarm_min, alarm_max = alarm_df_i['Raised Time'].min(), alarm_df_i['Raised Time'].max()
    print(f"{inverter} #alarms: {alarm_df_i.shape[0]}")
    
    data_min = alarm_min - pd.Timedelta(min_past_days, 'D')

    inv_df_i = inv_df[columns].copy()
#     print(inv_df_i.shape[0], len(set(inv_df_i['date'])))
    
    inv_df_i['date'] = pd.to_datetime(inv_df_i["date"])
    inv_df_i.rename(columns={'IN.GMRX.CHAR.'+inverter+'.Active Power (kW)': 'power',
                            'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)': 'temp1',
                            'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)': 'rad1',
                            'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)': 'temp2',
                            'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)': 'rad2'}, inplace=True)
    inv_df_i['hour'] = inv_df_i.date.dt.hour
#     inv_df_i = inv_df_i[(inv_df_i['date'] >= data_min) & (inv_df_i['date'] <= alarm_max)]
    # inv_df_i = inv_df_i[(inv_df_i.hour >= 6) & (inv_df_i.hour <= 18)]
    inv_df_i = inv_df_i.sort_values('date').reset_index(drop=True)
#     print(inv_df_i.shape[0], len(set(inv_df_i['date'])))
    
    min_date, max_date = inv_df_i.date.min(), inv_df_i.date.max()
    df_full = pd.DataFrame({TIMESTAMP_COL_NAME: pd.date_range(
                            start=min_date, end=max_date, freq="5min")}).merge(inv_df_i, on='date', how='left')
    print(df_full[TIMESTAMP_COL_NAME].isna().sum(), df_full['power'].isna().sum())
#     print(df_full.shape[0], len(set(df_full['date'])))
    
    # label generation - no impact of missing values since operation is happening on df_full
    # label-1: generation for next 5 minutes (the original time interval)
    # count how many alarms are generated till that time
    df_full['count_alarm'] = df_full['date'].apply(lambda x: np.sum(alarm_df_i['Raised Time'] <= x))
    
    # how many alarms are generated between the next time and the current time
    df_full['delta_alarm'] = df_full['count_alarm'].shift(-1) - df_full['count_alarm']
    
    # 1 if positive else 0 
    df_full['label'] = np.where(df_full['delta_alarm'] > 0, 1, 0)
#     print(df_full.shape[0], len(set(df_full['date'])))

    # count how many alarms are generated in next one hours - more realistic/actionworthy label
    df_full['count_alarm_1h'] = df_full['date'].apply(lambda x: np.sum(alarm_df_i['Raised Time'] <= x + pd.Timedelta(1, 'H')))
    df_full['delta_alarm_1h'] = df_full['count_alarm_1h'] - df_full['count_alarm']
    df_full['label_1h'] = np.where(df_full['delta_alarm_1h'] > 0, 1, 0)
    
    # count how many alarms are generated in next 24 hours - more realistic/actionworthy label
    df_full['count_alarm_24h'] = df_full['date'].apply(lambda x: np.sum(alarm_df_i['Raised Time'] <= x + pd.Timedelta(1, 'D')))
    df_full['delta_alarm_24h'] = df_full['count_alarm_24h'] - df_full['count_alarm']
    df_full['label_24h'] = np.where(df_full['delta_alarm_24h'] > 0, 1, 0)
    
    y = dict(df_full[label_col].value_counts())
    if 1 in y.keys():
        print('Initial True labels', y[1])

    # recalculate the hour and fill for the non-operational hours
    df_full['year'] = df_full.date.dt.year
    df_full['month'] = df_full.date.dt.month
    df_full['day'] = df_full.date.dt.day
    df_full['hour'] = df_full.date.dt.hour
    df_full['minute'] = df_full.date.dt.minute
    df_full.loc[(df_full['power'].isna()) & (df_full['hour'] > 18), 'power'] = 0
    df_full.loc[(df_full['power'].isna()) & (df_full['hour'] < 6), 'power'] = 0
    print(df_full[TIMESTAMP_COL_NAME].isna().sum(), df_full['power'].isna().sum())
#     print(df_full.shape[0], len(set(df_full['date'])))

    # impute data 
    if impute_data:
        df_imputed, fill_dates = impute_data(df_full, colname='power', method='backward')
        print(len(fill_dates), df_imputed['power'].isna().sum())
    else:
        fill_dates = []

    # add these dates to inverter data
    night_hours = df_full[(df_full['power'].isna()) & (df_full['hour'] > 18)]['date']
    morning_hours = df_full[(df_full['power'].isna()) & (df_full['hour'] < 6)]['date']

#     print(df_full.shape[0], len(set(df_full['date'])))
    # get the missing dates
    full_dates = df_full['date'].to_list()
    inv_dates = set(inv_df_i['date'].to_list() +\
                    night_hours.to_list() +\
                    morning_hours.to_list() +\
                    fill_dates
                   )

    last_na = None
    count_na = 0
    na_col = []
    for ii, dd in enumerate(full_dates):
        if dd not in inv_dates:
            last_na = ii
            count_na += 1
        if count_na > 0:
            na_col.append(ii-last_na)
        else:
            na_col.append(ii)

    # filter if minimum history is not available
    df_full['count_na'] = na_col
    df_full = df_full[df_full['count_na'] >= min_past_days * 24 * 12]
#     print(df_full.shape[0], len(set(df_full['date'])))
    
    y = dict(df_full[label_col].value_counts())
    if 1 in y.keys():
        print('... True labels (history)', y[1])
    
    # filter for operating hours
    df_full = df_full[(df_full.hour >= 6) & (df_full.hour <= 18)]
#     print(df_full.shape[0], len(set(df_full['date'])))
    y = dict(df_full[label_col].value_counts())
    if 1 in y.keys():
        print('... True labels (operation)', y[1])

    # filter for alarm timings
#     print(data_min, alarm_max, df_full.date.min(), df_full.date.max())
    df_full = df_full[df_full.date >= data_min]
    df_full = df_full[df_full.date <= alarm_max]
    y = dict(df_full[label_col].value_counts())
    if 1 in y.keys():
        print('... True labels (alarms)', y[1])
    
#     print(df_full['label'].value_counts())
#     print(df_full.shape[0], len(set(df_full['date'])))
    x = df_full[['date', 'label', 'label_1h', 'label_24h']]
#     print(x.shape[0], len(set(x['date'])))
    
    feature_dfs[inverter] = x
    print(f"{inverter} #alarms: {alarm_df_i.shape[0]}, #data points: {x.shape[0]}")
    
    

In [ ]:
# dfx = df_full[['date', 'count_alarm', 'delta_alarm', 'label']]
# dfx = df_full[['date', 'count_alarm_1h', 'delta_alarm_1h', 'label_1h']]
dfx = df_full[['date', 'count_alarm_24h', 'delta_alarm_24h', 'label_24h']]

In [ ]:
alarm_df_i['Raised Time']

In [ ]:
dfx[dfx['date']=='2020-09-17 18:55:00']

In [ ]:
dfx[dfx.label_24h==1]

In [ ]:
dfx.plot(x='date')

In [ ]:
dfx['label_1h'].value_counts()

In [ ]:
pd.to_datetime('2020-12-23 23:50:00') + pd.Timedelta(1, 'H')

In [ ]:
dfx['yearmo'] = dfx['date'].apply(lambda x: f"{x.year}{x.month:02d}")
dfx['yearmo'].value_counts()

In [ ]:
dfx[dfx.yearmo=='201912'].plot(x='date')

In [ ]:
with open('inverter-labels-v3.pkl', 'wb') as handle:
    pickle.dump(feature_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

| File | Description |
| --- | --- |
|'inverter-labels-v3.pkl' | no data imputation, 1h and 24h label included
|'inverter-labels-v2.pkl' | with data imputation
|'inverter-labels.pkl' | no data imputation

In [ ]:
df_full['year'] = df_full.date.dt.year
df_full['month'] = df_full.date.dt.month
df_plot = df_full[(df_full.year==2019) & (df_full.month==5)]

In [ ]:
df_plot['label'].value_counts()

In [ ]:
df_plot[['date', 'label']].plot(x='date')

In [ ]:
df_plot['label_24h'].value_counts()

In [ ]:
df_plot[['date', 'label_24h']].plot(x='date')

In [ ]:
df_plot[['date', 'label', 'label_24h']].plot(x='date')
# plt.plot(df_full['date'], df_full['label'])
# plt.plot(df_full['date'], df_full['label_24h'], alpha=0.5)

In [ ]:
duplicate = inv_df_i['date'][inv_df_i['date'].duplicated()] 
duplicate

In [ ]:
duplicate = df_full['date'][df_full['date'].duplicated()] 
duplicate.dt.month.value_counts()

In [ ]:
inv_df_i[inv_df_i['date'] == '2019-01-08 00:00:00']

In [ ]:
data_dir = 'data/Inverter'
file = 'min190808.csv'
df_ = pd.read_csv(os.path.join(data_dir, file))
df_[['Time', 'IN.GMRX.CHAR.INV-01.Active Power (kW)', 
     'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)',
     'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)',
     'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)',
     'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)']]

In [ ]:
file = 'min190902.csv'
df_ = pd.read_csv(os.path.join(data_dir, file))
df_[['Time', 'IN.GMRX.CHAR.INV-01.Active Power (kW)', 
     'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)',
     'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)',
     'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)',
     'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)']]

In [ ]:
from IPython.display import display, HTML
cols = ['Time', 'IN.GMRX.CHAR.INV-01.Active Power (kW)', 
     'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)',
     'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)',
     'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)',
     'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)']
for file in tqdm(os.listdir(data_dir)):
    try:
        df_ = pd.read_csv(os.path.join(data_dir, file))
        df_["date"] = pd.to_datetime(df_["Time"])
        dfx = df_[df_['date'] == '2019-01-09 00:00:00']
        if dfx.shape[0] > 0:
            print(file)
            display(dfx[cols])
    except:
        print("problem", file)


In [ ]:
pd.to_datetime('08-Jan-19 00:00'), pd.to_datetime('01-08-2019 00:00')

In [ ]:
pd.to_datetime('01-08-2019 00:00', format='%d-%m-%Y %H:%M')

In [ ]:
pd.to_datetime('13-Jan-19 00:00'), pd.to_datetime('01-13-2019 00:00')

In [ ]:
s = 'min190801.csv'
s[3:7]

In [ ]:
inverter_name = 'INV-509'
num_past_days = 1
features = ['IN.GMRX.CHAR.'+inverter_name+'.Active Power (kW)', 
#             'IN.GMRX.CHAR.WS-20 MW.Ambient Temperature (°C)', # all nulls
            'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)',
            'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)',
#             'IN.GMRX.CHAR.WS-5 MW.Ambient Temperature (°C)',  # all nulls
            'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)',
            'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)'
           ]
columns = ['date'] + features
inverter_index = inverters.index(inverter_name)
alarm_df_i = alarm_df[alarm_df["Controller Name"] == inverters[inverter_index]].copy()
alarm_df_i['Raised Time'] = pd.to_datetime(alarm_df_i['Raised Time'])
alarm_df_i = alarm_df_i.sort_values('Raised Time').reset_index(drop=True)
print("Number of alarms:", alarm_df_i.shape[0])

TIMESTAMP_COL_NAME = 'date'
POWER_COL = 'power'
inv_df_i = inv_df[columns].copy()
inv_df_i['date'] = pd.to_datetime(inv_df_i["date"])
inv_df_i.rename(columns={'IN.GMRX.CHAR.'+inverter_name+'.Active Power (kW)': 'power',
                        'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)': 'temp1',
                        'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)': 'rad1',
                        'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)': 'temp2',
                        'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)': 'rad2'}, inplace=True)
inv_df_i['hour'] = inv_df_i.date.dt.hour
# inv_df_i = inv_df_i[(inv_df_i.hour >= 6) & (inv_df_i.hour <= 18)]
inv_df_i = inv_df_i.sort_values('date').reset_index(drop=True)

print('Missing time:', inv_df_i[TIMESTAMP_COL_NAME].isna().sum())
print('Missing power:', inv_df_i['power'].isna().sum())
inv_df_i


In [ ]:
min_date, max_date = inv_df_i.date.min(), inv_df_i.date.max()
# min_date, max_date = alarm_df_i['Raised Time'].min() - pd.Timedelta(7, 'D'), alarm_df_i['Raised Time'].max()
print(min_date, max_date)

df_full = pd.DataFrame({TIMESTAMP_COL_NAME: pd.date_range(
                        start=min_date, end=max_date, freq="5min")}).merge(inv_df_i, on='date', how='left')
print(df_full[TIMESTAMP_COL_NAME].isna().sum())
print(df_full['power'].isna().sum())
df_full.head(100)

In [ ]:
df_full['hour'] = df_full['date'].dt.hour
# df_full[(df_full['hour'] > 18) & (df_full['hour'] < 6) & (df_full['power'].isna())]
# df_full[(df_full['power'].isna()) & (df_full['hour'] > 18) & (df_full['hour'] < 6)]
# df_full[(df_full['power'].isna())]['hour'].value_counts()
# df_full[(df_full['power'].isna()) & (df_full['hour'] > 18)]['hour'].value_counts()
df_full.loc[(df_full['power'].isna()) & (df_full['hour'] > 18), 'power'] = 0
df_full.loc[(df_full['power'].isna()) & (df_full['hour'] < 6), 'power'] = 0

In [ ]:
df_full['power'].isna().sum()

In [ ]:
df_full['hour'] = df_full['date'].dt.hour
night_hours = df_full[(df_full['power'].isna()) & (df_full['hour'] > 18)]['date']
morning_hours = df_full[(df_full['power'].isna()) & (df_full['hour'] < 6)]['date']

In [ ]:
night_hours.to_list()

### Get the number of missing days (based on "power")

In [ ]:
df_full[['date', 'power']].plot(x='date')

In [ ]:
full_dates = df_full['date'].to_list()
inv_dates = set(inv_df_i['date'].to_list())

last_na = None
count_na = 0
na_col = []
for ii, dd in enumerate(full_dates):
    if dd not in inv_dates:
        last_na = ii
        count_na += 1
    if count_na > 0:
        na_col.append(ii-last_na)
    else:
        na_col.append(ii)


In [ ]:
df_full['count_na'] = na_col
df_full[['date', 'count_na']].plot(x='date')

In [ ]:
import sys

def get_missing_data(current_date, df_, col='power'):
#     print(current_date)
    df_ = df_.sort_values('date').reset_index(drop=True)
    df__ = df_[df_['date'] <= current_date]
    if df__[col].isna().sum() == 0:
        return 0
    else:
        last_na_date = df__[df__[col].isna()]['date'].max()
        last_index = df__.index[df__[col].isna()].max()
        return df__.index.max() - last_index

def get_missing_(x):
    if np.isnan(x).sum() == 0:
        return 0
    else:
#         print(x)
#         print(np.isnan(x))
#         print(np.argwhere(np.isnan(x)))
        last_index = np.argwhere(np.isnan(x)).max()
#         print(len(x) - last_index - 1)
        return len(x) - 1 - last_index
#         sys.exit('HERE')

In [ ]:
# df_full['count_na'] = df_full['date'].apply(lambda x: get_missing_data(x, df_full))
# df_full['count_na'] = df_full['power'].expanding(min_periods=1).apply(lambda x: np.isnan(x).sum(), raw=False)
# df_full['count_na'] = df_full['power'].expanding(min_periods=1).apply(lambda x: get_missing_(x), raw=False)
df_full['count_na'] = df_full['power'].rolling(window = 7*24*12).apply(lambda x: np.isnan(x).sum())

In [ ]:
df_full['count_na'].isna().sum(), df_full.shape

In [ ]:
df_full[['date', 'count_na']].plot(x='date')

In [ ]:
last_na = None
count_na = 0
na_col = []
for ii, row in tqdm(df_full.iterrows()):
    if np.isnan(row['power']):
        last_na = ii
        count_na += 1
    if count_na > 0:
        na_col.append(ii-last_na)
    else:
        na_col.append(ii)


In [ ]:
row['power'], type(row['power']), row['power'] == np.datetime64('NaT')

In [ ]:
print(df_full['power'].isna().sum())

In [ ]:
df_full['power'][209]

In [ ]:
df_full['count_na2'] = na_col

In [ ]:
df_full[['date', 'count_na', 'count_na2']].plot(x='date')

In [ ]:
# label generation
df_full['count_alarm'] = df_full['date'].apply(lambda x: np.sum(alarm_df_i['Raised Time'] <= x))
df_full['delta_alarm'] = df_full['count_alarm'].shift(-1) - df_full['count_alarm']
df_full['label'] = df_full['delta_alarm'] > 0

In [ ]:
df_full['label'].value_counts(True)

In [ ]:
df_full['count_na2'].describe()

In [ ]:
df_full2 = df_full[df_full['count_na2'] >= 7 * 24 * 12]
df_full2['label'].value_counts(True)

In [ ]:
df_full2[['date', 'power']].plot(x='date')

In [ ]:
def get_missing_dates(df):
    last_na = None
    count_na = 0
    na_col = []
    for ii, row in tqdm(df.iterrows()):
#         print(ii, row['power'])
        if np.isnan(row['power']):
            last_na = ii
            count_na += 1
        if count_na > 0:
            na_col.append(ii-last_na)
        else:
            na_col.append(ii)
    return na_col

In [ ]:
y = dict(df_full['label'].value_counts())
True in y.keys()

In [ ]:
alarm_df_i

## Get all the missing dates

In [ ]:
TIMESTAMP_COL_NAME = 'date'
POWER_COL = 'power'
for inverter in inverters:

    features = ['IN.GMRX.CHAR.'+inverter_name+'.Active Power (kW)', 
    #             'IN.GMRX.CHAR.WS-20 MW.Ambient Temperature (°C)', # all nulls
                'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)',
                'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)',
    #             'IN.GMRX.CHAR.WS-5 MW.Ambient Temperature (°C)',  # all nulls
                'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)',
                'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)'
               ]
    columns = ['date'] + features
    inv_df_i = inv_df[columns].copy()
    inv_df_i['date'] = pd.to_datetime(inv_df_i["date"])
    inv_df_i.rename(columns={'IN.GMRX.CHAR.'+inverter_name+'.Active Power (kW)': 'power',
                            'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)': 'temp1',
                            'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)': 'rad1',
                            'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)': 'temp2',
                            'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)': 'rad2'}, inplace=True)
    inv_df_i['hour'] = inv_df_i.date.dt.hour
    inv_df_i = inv_df_i.sort_values('date').reset_index(drop=True)

    min_date, max_date = inv_df_i.date.min(), inv_df_i.date.max()
    df_full = pd.DataFrame({TIMESTAMP_COL_NAME: pd.date_range(
                            start=min_date, end=max_date, freq="5min")}).merge(inv_df_i, on='date', how='left')
#     print(df_full[TIMESTAMP_COL_NAME].isna().sum())
    count_na = df_full['power'].isna().sum()
    print(f"{inverter}- #missing days {count_na}")


In [ ]:
df_full['date'][df_full['power'].isna()].apply(lambda x: x.year).value_counts()

In [ ]:
def fill_timegaps(df_sub, min_date, max_date):
    """Fill time gaps within a time-series dataframe.
    """
    min_date = df_sub[TIMESTAMP_COL_NAME].min()
    timespan = pd.DataFrame(
        {
            TIMESTAMP_COL_NAME: pd.date_range(
                start=min_date, end=max_date, freq="D"),
            GRAIN_COL_NAMES[0]: df_sub.iloc[0][GRAIN_COL_NAMES[0]],
            GRAIN_COL_NAMES[1]: df_sub.iloc[0][GRAIN_COL_NAMES[1]]
        })
    df_sub = pd.merge(
        df_sub,
        timespan,
        on=[TIMESTAMP_COL_NAME],
        how="right")
    df_sub.sort_values(
        [TIMESTAMP_COL_NAME]).reset_index(drop=True)
    return df_sub

TIMESTAMP_COL_NAME = 'date'
fill_timegaps(inv_df_i, inv_df_i.date.min(), inv_df_i.date.max())

In [ ]:
start_date, end_date = '2019-07-01', '2019-11-01'
mask = (inv_df_i['date'] > start_date) & (inv_df_i['date'] <= end_date)
df_plot = inv_df_i.loc[mask]
df_plot[['date', 'power']].plot(x='date', kind='line')

In [ ]:
inv_df_i.date.min(), alarm_df_i['Raised Time'].min()

In [ ]:
inv_df_i.date.max()

In [ ]:
inv_df_i['count_alarm'] = inv_df_i['date'].apply(lambda x: np.sum(alarm_df_i['Raised Time'] <= x))
# np.sum(alarm_df_i['Raised Time'] <= inv_df_i['date'][261056])
inv_df_i['label'] = inv_df_i['count_alarm'].shift(-1) - inv_df_i['count_alarm'] 

In [ ]:
inv_df_i['label'].value_counts()

In [ ]:
inv_df_i[inv_df_i['label'] == 27]

In [ ]:
inv_df_i.loc[71802]

In [ ]:
# np.sum(alarm_df_i['Raised Time'] <= inv_df_i['date'][71802])
# alarm_df_i[alarm_df_i['Raised Time'] <= pd.to_datetime('2019-09-08 18:55:00')]
alarm_df_i[alarm_df_i['Raised Time'] <= pd.to_datetime('2019-10-01 06:00:00')]

In [ ]:
alarm_df_i.head()

In [ ]:
def rolling_features(df, start_step, window_size, funcs):
    features = df.shift(start_step).rolling(window_size, min_periods=window_size).agg(funcs)
    features.columns = ["{}_{}{}".format(x[0], x[1], str(window_size)) for x in features.columns]
    return features

def create_features(df, colnames, ROLLING_WINDOWS):
    # Feature engineering
    df["day"] = df[TIMESTAMP_COL_NAME].apply(lambda x: x.day)
    df["dayofweek"] = df[TIMESTAMP_COL_NAME].apply(lambda x: x.dayofweek)
    df["weekofyear"] = df[TIMESTAMP_COL_NAME].apply(lambda x: x.isocalendar()[1])
    df["month"] = df[TIMESTAMP_COL_NAME].apply(lambda x: x.month)

    # assume each record is the first day of forecast period so shift rolling calcs back by 1
    start_step = 1 
    for col in colnames:
        for window in ROLLING_WINDOWS:
            feats = rolling_features(df[[col]], start_step=1, window_size=window, funcs=[np.mean, np.std]).reset_index(drop=True)
            df = pd.concat([df, feats], axis=1)
    return df

In [ ]:
# we will create rolling calculation sof sum and std with these window lengths  
# since every data point is at 5 minutes interval, 1 day = 12 * 24 data points
windows = [x*12*24 for x in [1, 7, 14, 21, 30]] 
df_ = create_features(inv_df_i, colnames=['power'], ROLLING_WINDOWS=windows)

In [ ]:
df_

In [ ]:
def generate_ts_features(df, alarms, features, inp_window, tgt_window):
    dmin, dmax = df_['date'].min(), df_['date'].max()
    nestim = 0
    df = df.sort_values(by=['date']).reset_index(drop=True)
    n = df.shape[0]
    feature_df = {"x": [], "y": [], "date": []}
    for ind in tqdm(range(n-inp_window-tgt_window)):
#     for ind in range(n-inp_window+1):
        df_i = df.iloc[ind: ind + inp_window]
        x = df_i[features].values
        df_o = df.iloc[ind + inp_window : ind + inp_window + tgt_window]
        
        y = 0
        # check if any alarm falls within the output window
        for jj in alarms["Raised Time"]:
            if jj >= df_o['date'].min() and jj <= df_o['date'].max():
                y = 1
                break
#         print(df_i['date'].min(), df_i['date'].max())
#         print(y_tentative)
        date_i = df_i["date"].tolist()[-1]
        feature_df["x"].append(x)
        feature_df["y"].append(y)
        feature_df["date"].append(date_i)
    return pd.DataFrame(feature_df)


In [ ]:
features = ['power', 'temp1', 'rad1', 'temp2', 'rad2', 'hour', 'day',
           'dayofweek', 'weekofyear', 'month', 'power_mean288', 'power_std288',
           'power_mean2016', 'power_std2016', 'power_mean4032', 'power_std4032',
           'power_mean6048', 'power_std6048', 'power_mean8640', 'power_std8640']

# 1 day = 12 hours if restriced to morning 6 AM to evening 6 PM
# each hour = 12 data points (@5 minutes)
inp_window = 2 * 12 * 12
tgt_window = 1 * 12 * 12
generate_ts_features(df_, alarm_df_i, features, inp_window, tgt_window)

In [ ]:
alarm_df_i.shape, df_.shape

In [ ]:
1637/258954

In [ ]:
# alarm data cannot be merged with inverter data because they are at different granular levels
label_df = df_[["date"]].merge(alarm_df_i[['Raised Time']], left_on='date', right_on='Raised Time', how='left').\
           rename(columns={'Raised Time': 'label'})
label_df['label'].isna().sum()

In [ ]:
alarm_df_i

In [ ]:
df2 = inv_df_i[['date', 'power']].copy()
df2 = df2[df2.date.dt.date == pd.to_datetime('2020-12-23')].reset_index(drop=True)
df2.plot(x='date')

In [ ]:
f = df2.rolling(12).agg(np.std).reset_index(drop=True)
f.columns = ['agg']
df2 = pd.concat([df2, f], axis=1)
df2.plot(x='date')

In [ ]:
alarms = []
last_alarm = False
count = 0
for _, row in alarm_df_i.iterrows():
    if last_alarm:
        dt = row['Raised Time'] - last_alarm
        if dt.total_seconds()/60 > 5:
            alarms.append((row['Raised Time'], row['Error Code']))
        else:
#             print(last_alarm, row['Raised Time'])
            count += 1
    last_alarm = row['Raised Time']
print(len(alarms), count)

In [ ]:
alarms

In [ ]:
count = 0
all_df = []
for inverter in inverters:
    
    alarm_df_i = alarm_df[alarm_df["Controller Name"] == inverter].copy()
    alarm_df_i['Raised Time'] = pd.to_datetime(alarm_df_i['Raised Time'])
    alarm_df_i = alarm_df_i.sort_values('Raised Time').reset_index(drop=True)

    df_t = alarm_df_i[['Raised Time']].copy()
    df_t['shifted'] = df_t['Raised Time'].shift(-1)
    df_t['diff'] = df_t['shifted'] - df_t['Raised Time']
    df_t['diff'] = df_t['diff'].apply(lambda x: x.total_seconds()/60)
    print(f"{inverter}- #alarms {alarm_df_i.shape[0]}, actual {np.sum(df_t['diff'] < 5)}")
    count += np.sum(df_t['diff'] < 5)
    all_df.append(df_t)
print(count)

In [ ]:
all_df = pd.concat(all_df, axis=0)
all_df['diff'].describe()

In [ ]:
alarm_df.shape

In [ ]:
# n, bins, patches = plt.hist(x, num_bins, facecolor='blue', alpha=0.5)

n, bins, patches = plt.hist(all_df['diff'], 
                            bins=[1.16666667e-01, 1, 2, 3, 4, 5, 10, 
                                  2.81441683e+04, 5.62882200e+04, 8.44322717e+04,
                                    1.12576323e+05, 1.40720375e+05, 1.68864427e+05,
                                    5.62881150e+05], facecolor='blue', alpha=0.5)
# sns.histplot(all_df['diff'], bins=20)
plt.show()


In [ ]:
inverter = 'INV-16'
code_map = {7006: 1, 7501: 2, 7502: 3, 3511: 4, 3504: 5, 6448: 6, 7704: 7, 1500: 0}

for inverter in inverters:
    alarm_df_i = alarm_df[alarm_df["Controller Name"] == inverter].copy()
    alarm_df_i['Raised Time'] = pd.to_datetime(alarm_df_i['Raised Time'])
    alarm_df_i = alarm_df_i.sort_values('Raised Time').reset_index(drop=True)
    alarm_df_i['scaled'] = alarm_df_i['Error Code'].map(code_map)
    alarm_df_i[['Raised Time', 'scaled']].plot(x='Raised Time', title=f"{inverter}, total alarms: {alarm_df_i.shape[0]}")
    print(inverter, alarm_df_i['Error Code'].value_counts())

In [ ]:
len(inverters)

In [ ]:
alarm_df['Error Code'].value_counts()